In [90]:
from math import log
from sklearn.metrics.pairwise import cosine_similarity as cos_sim
%run Summary_Processes/Generic_Summarizer.ipynb
import numpy as np

class TextRank_Jaccard_process :
    
    def __init__(self, a = 1, b = 1, weighted=False, method = "tr", lsanbcompfun = None, diag = "none", bias = 0) :
        #Assymetric coefficients
        self.a = a
        self.b = b
        
        #Parameters to calculate the
        self.weighted = weighted
        self.method = method
        self.lsanbcompfun = lsanbcompfun # ça c'est du nom de variable explicite.
        self.diag = diag
        self.bias = bias
        
        methodstr = "TextRank" if method == "tr" else "LSA"
        weightedstr = "weighted" if weighted else "unweighted"
        self.__name__ = methodstr+"_Jaccard_process("+str(a)+", "+str(b)+", "+weightedstr+","+diag+","+str(bias)+")"
    
    def preprocess(self, corpus):
        pass
    
    def summarize(self, corpus) :
        
        #Calculating the similarity
        self.sent_words = []
        for sent in corpus :
            self.sent_words.append(set(sent.split()))
        
        sen_length = [log(len(corpus[i].split())) for i in range(len(corpus))]
        l = len(corpus)
        m = np.array([[0 for i in range(j)]+[len(self.sent_words[i].intersection(self.sent_words[j])) for i in range(j,l)]
                      for j in range(l)])

        m += m.T
        X = np.divide(m, np.array([[sen_length[i] + sen_length[j] for i in range(l)] for j in range(l)]))
        
        #Calculte the assymetric part 
        dist = np.array([np.arange(X.shape[0]) - i for i in range(0,X.shape[0])])
        pos = dist > 0
        neg = dist < 0
        factor = (np.power(self.a, np.abs(np.multiply(dist,pos)))
                  + np.power(self.b, np.abs(np.multiply(dist,neg)))
                  - 1)
        
        #Modify the similarity according to the assymetry
        matrix = np.multiply(factor,cos_sim(X, X))
        
        return generic_summarizer(self.method, matrix, corpus, self.weighted, self.lsanbcompfun, diag =  self.diag, bias = self.bias)